In [12]:
import numpy as np
import pandas as pd
import warnings


class AHP:
    def __init__(self, criteria, factors):
        self.RI = (0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49)
        self.criteria = criteria  # Criterion
        self.factors = factors  # factor
        self.num_criteria = criteria.shape[0]
        self.num_factors = factors[0].shape[0]

    def cal_weights(self, input_matrix):
        input_matrix = np.array(input_matrix)
        n, n1 = input_matrix.shape
        assert n == n1, 'not a square'
        for i in range(n):
            for j in range(n):
                if np.abs(input_matrix[i, j] * input_matrix[j, i] - 1) > 1e-7:
                    raise ValueError('is not an anti-mutually symmetric matrix')

        eigenvalues, eigenvectors = np.linalg.eig(input_matrix)

        max_idx = np.argmax(eigenvalues)
        max_eigen = eigenvalues[max_idx].real
        eigen = eigenvectors[:, max_idx].real
        eigen = eigen / eigen.sum()

        if n > 9:
            CR = None
            warnings.warn('Unable to judge consistency')
        else:
            CI = (max_eigen - n) / (n - 1)
            CR = CI / self.RI[n]
        return max_eigen, CR, eigen

    def run(self):
        max_eigen, CR, criteria_eigen = self.cal_weights(self.criteria)
        print('Criterion{} Layer: Largest Eigenvalue{:<5f},CR={:<5f},examine{}ok'.format(
            max_eigen, CR, '' if (CR < 0.1 or CR=='nan' )else 'no'))
        print('Criterion layer weights={}\n'.format(criteria_eigen))

        max_eigen_list, CR_list, eigen_list = [], [], []
        k = 1
        for i in self.factors:
            max_eigen, CR, eigen = self.cal_weights(i)
            max_eigen_list.append(max_eigen)
            CR_list.append(CR)
            eigen_list.append(eigen)
            print('Criterion {} Layer: Largest Eigenvalue{:<5f},CR={:<5f},examine{}ok'.format(
                k, max_eigen, CR, '' if (CR < 0.1 or CR==None) else 'no'))
            print('factor layer weights={}\n'.format(eigen))

            k = k + 1

        return criteria_eigen, eigen_list


In [16]:
def main():
    # 准则重要性矩阵
    criteria = np.array([[1, 1/5, 5],
                         [5, 1, 9],
                         [1/5, 1/9, 1]
                         ])

    # 对每个准则，方案优劣排序
    b1 = np.array([[1, 1/5], [5, 1]])

    b2 = np.array([[1,1],[1,1]])

    b3= np.array([[1,1],[1,1]])

    b = [b1, b2, b3]
    a, c = AHP(criteria, b).run()
    #下一段将用到此函数
    #fuzzy_eval(a,c)


if __name__ == '__main__':

    main()


准则层：最大特征值3.117100,CR=0.065055,检验通过
准则层权重=[0.20669538 0.73519341 0.05811121]

准则 1 因素层：最大特征值2.000000,CR=0.000000,检验通过
因素层权重=[0.16666667 0.83333333]

准则 2 因素层：最大特征值2.000000,CR=0.000000,检验通过
因素层权重=[0.5 0.5]

准则 3 因素层：最大特征值2.000000,CR=0.000000,检验通过
因素层权重=[0.5 0.5]

